In [24]:
# PyData
import pandas
import numpy
from pandas import DataFrame, Series
from sklearn import datasets, linear_model, preprocessing, cross_validation
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor
from sklearn.linear_model import LinearRegression, Ridge

# System
import datetime
import os
import math

# Graphing
#%matplotlib inline # Only works on Python 3 in the docker container
#import seaborn # Only works on Python 3 in the docker container

#os.environ['PYSPARK_PYTHON'] = 'python2'

%matplotlib inline

# Spark
import pyspark
from pyspark.sql import SQLContext

from pyspark.ml import Pipeline
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.feature import VectorIndexer
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.mllib.util import MLUtils
from pyspark.mllib.regression import LabeledPoint

sc = pyspark.SparkContext('local[8]')
sqlContext = SQLContext(sc)

ValueError: Cannot run multiple SparkContexts at once; existing SparkContext(app=pyspark-shell, master=local[8]) created by __init__ at <ipython-input-1-eee5e5da39a2>:33 

In [25]:
df_basic = DataFrame.from_csv("train_features_basic.csv", index_col=['Date', 'Store'])
df_means = DataFrame.from_csv("train-features-predicted_mean.csv", index_col=['Date', 'Store'])

In [26]:
df_sales = DataFrame.from_csv("train.csv", index_col=['Date', 'Store'])[['Sales']]

/opt/conda/lib/python3.4/site-packages/pandas/io/parsers.py:1170: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  data = self._reader.read(nrows)


In [27]:
df_features = df_means.join(df_basic)
train_df = df_sales.join(df_features)
train_df.head()

Sales  Sales_predicted  DayOfWeek  Open  Promo  \
Date       Store                                                   
2015-07-31 1       5263      5145.283582          5     1      1   
           2       6064      5115.882353          5     1      1   
           3       8314      8138.089552          5     1      1   
           4      13995     10275.776119          5     1      1   
           5       4822      5308.835821          5     1      1   

                  StateHoliday  SchoolHoliday  cal:dayOfMonth  cal:dayOfWeek  \
Date       Store                                                               
2015-07-31 1                 0              1              31              4   
           2                 0              1              31              4   
           3                 0              1              31              4   
           4                 0              1              31              4   
           5                 0              1              31              4   

                  cal:dayofyear  cal:month  cal:quarter  cal:weekofyear  \
Date       Store                                                          
2015-07-31 1                212          7            3              31   
           2                212          7            3              31   
           3                212          7            3              31   
           4                212          7            3              31   
           5                212          7            3              31   

                  cal:year  
Date       Store            
2015-07-31 1          2015  
           2          2015  
           3          2015  
           4          2015  
           5          2015

In [28]:
del df_features
del df_basic
del df_means
del df_sales

In [29]:
train_df['Sales_predicted'] = train_df['Sales_predicted'].fillna(train_df['Sales_predicted'].mean())

In [30]:
train_df['Sales_predicted'] = train_df['Sales_predicted'].map(lambda x: int(x))

In [31]:
train_df.reset_index(inplace=True)

,Date,Store,Sales,Sales_predicted,DayOfWeek,Open,Promo,StateHoliday,SchoolHoliday,cal:dayOfMonth,cal:dayOfWeek,cal:dayofyear,cal:month,cal:quarter,cal:weekofyear,cal:year
0,2015-07-31,1,5263,5145,5,1,1,0,1,31,4,212,7,3,31,2015
1,2015-07-31,2,6064,5115,5,1,1,0,1,31,4,212,7,3,31,2015
2,2015-07-31,3,8314,8138,5,1,1,0,1,31,4,212,7,3,31,2015
3,2015-07-31,4,13995,10275,5,1,1,0,1,31,4,212,7,3,31,2015
4,2015-07-31,5,4822,5308,5,1,1,0,1,31,4,212,7,3,31,2015


In [32]:
del train_df['Date']

In [77]:

train_df.head()

,Sales,Store,Sales_predicted,DayOfWeek,Open,Promo,StateHoliday,SchoolHoliday,cal:dayOfMonth,cal:dayOfWeek,cal:dayofyear,cal:month,cal:quarter,cal:weekofyear,cal:year
0,5263,1,5145,5,1,1,0,1,31,4,212,7,3,31,2015
1,6064,2,5115,5,1,1,0,1,31,4,212,7,3,31,2015
2,8314,3,8138,5,1,1,0,1,31,4,212,7,3,31,2015
3,13995,4,10275,5,1,1,0,1,31,4,212,7,3,31,2015
4,4822,5,5308,5,1,1,0,1,31,4,212,7,3,31,2015


In [ ]:
# Sales Predicted Difference %

In [82]:
train_df['Sales_mean_prediction_error'] = (train_df['Sales_predicted'] - train_df['Sales']) / train_df['Sales']

In [ ]:
del train_df['Sales_predicted']

In [84]:
feature_columns = list(train_df.columns)
feature_columns.remove("Sales")
feature_columns = ['Sales'] + feature_columns
train_df = train_df[feature_columns]

In [85]:

train_df.head()

,Sales,Store,DayOfWeek,Open,Promo,StateHoliday,SchoolHoliday,cal:dayOfMonth,cal:dayOfWeek,cal:dayofyear,cal:month,cal:quarter,cal:weekofyear,cal:year,Sales_mean_prediction_error
0,5263,1,5,1,1,0,1,31,4,212,7,3,31,2015,-0.022421
1,6064,2,5,1,1,0,1,31,4,212,7,3,31,2015,-0.156497
2,8314,3,5,1,1,0,1,31,4,212,7,3,31,2015,-0.021169
3,13995,4,5,1,1,0,1,31,4,212,7,3,31,2015,-0.265809
4,4822,5,5,1,1,0,1,31,4,212,7,3,31,2015,0.100788


In [86]:
df = sqlContext.createDataFrame(train_df)
# This relies on correct order: [0] is sales, [1:] are all features
df = df.map(lambda row: LabeledPoint(row[0], row[1:])).toDF()


In [ ]:

featureIndexer = VectorIndexer(inputCol="features", outputCol="indexedFeatures", maxCategories=2000).fit(df)
# 0.28756570690198635 with maxCat=4

(trainingData, testData) = df.randomSplit([0.7, 0.3])


# Train a RandomForest model.
rf = RandomForestRegressor(numTrees=15, maxDepth=5, maxBins=1200, featuresCol="indexedFeatures")

# Chain indexer and forest in a Pipeline
pipeline = Pipeline(stages=[featureIndexer, rf])


In [ ]:

# Train model.  This also runs the indexer.
model = pipeline.fit(trainingData)



In [ ]:

# Make predictions.
predictions = model.transform(testData)

# Select example rows to display.
predictions.select("prediction", "label", "features").show(5)

# Select (prediction, true label) and compute test error
evaluator = RegressionEvaluator(
    labelCol="label", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(predictions)
# print "Root Mean Squared Error (RMSE) on test data = %g" % rmse

# Compute RMPSE
squares = predictions.rdd.filter(lambda x: x.label != 0).map(lambda x: ((x.label - x.prediction) / x.label) *  ((x.label - x.prediction) / x.label))

mean = squares.mean()
import math
math.sqrt(mean)

In [17]:

math.sqrt(mean)

0.3473766656723991

In [18]:
# Eval

In [69]:
df_basic = DataFrame.from_csv("test_features_basic.csv", index_col=['Date', 'Store'])
df_means = DataFrame.from_csv("test-features-predicted_mean.csv", index_col=['Date', 'Store'])

In [61]:
del df_means['Unnamed: 0']

In [62]:
df_test_features = df_means.join(df_basic)

In [63]:
#del df_test_features['Id']
# XXX WE NEED THIS LATER BUT NEED TO REMOVE BEFORE GIVING TO SPARK?

In [64]:
df_test_features.reset_index(inplace=True)
del df_test_features['Date']

In [66]:
df_test_features.head()

,Store,Sales_predicted,Id,DayOfWeek,Open,Promo,StateHoliday,SchoolHoliday,cal:dayOfMonth,cal:dayOfWeek,cal:dayofyear,cal:month,cal:quarter,cal:weekofyear,cal:year
0,1,4961.029412,1,4,1,1,0,0,17,3,260,9,3,38,2015
1,3,7994.426471,2,4,1,1,0,0,17,3,260,9,3,38,2015
2,7,9406.309859,3,4,1,1,0,0,17,3,260,9,3,38,2015
3,8,7194.929577,4,4,1,1,0,0,17,3,260,9,3,38,2015
4,9,6777.617647,5,4,1,1,0,0,17,3,260,9,3,38,2015


In [55]:
len(df_test_features.columns)

14

In [56]:
df_test = sqlContext.createDataFrame(df_test_features)
# This relies on correct order: [0] is sales, [1:] are all features
df_test = df_test.map(lambda row: LabeledPoint(0, features=row[0:])).toDF()

In [57]:
df_test.take(5)

[Row(features=DenseVector([1.0, 4961.0294, 4.0, 1.0, 1.0, 0.0, 0.0, 17.0, 3.0, 260.0, 9.0, 3.0, 38.0, 2015.0]), label=0.0),
 Row(features=DenseVector([3.0, 7994.4265, 4.0, 1.0, 1.0, 0.0, 0.0, 17.0, 3.0, 260.0, 9.0, 3.0, 38.0, 2015.0]), label=0.0),
 Row(features=DenseVector([7.0, 9406.3099, 4.0, 1.0, 1.0, 0.0, 0.0, 17.0, 3.0, 260.0, 9.0, 3.0, 38.0, 2015.0]), label=0.0),
 Row(features=DenseVector([8.0, 7194.9296, 4.0, 1.0, 1.0, 0.0, 0.0, 17.0, 3.0, 260.0, 9.0, 3.0, 38.0, 2015.0]), label=0.0),
 Row(features=DenseVector([9.0, 6777.6176, 4.0, 1.0, 1.0, 0.0, 0.0, 17.0, 3.0, 260.0, 9.0, 3.0, 38.0, 2015.0]), label=0.0)]

In [70]:
prediction = df_test_features[['Id']]

In [73]:
prediction['Sales'] = model.transform(df_test).map(lambda x: x.prediction).collect()

/opt/conda/lib/python3.4/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [75]:
prediction.head()

,Id,Sales
0,1,5553.403629
1,2,7631.575947
2,3,9618.671132
3,4,7434.123542
4,5,6623.967844


In [76]:

prediction.to_csv( "spark-v1.csv", index = False )

In [79]:
model.transform(trainingData).map(lambda x: x.prediction).take(5)

[5634.365778669672,
 8519.252856126777,
 6260.578432569513,
 10105.681959690635,
 6133.313960197123]

In [81]:
 trainingData.take(5)

[Row(features=DenseVector([2.0, 5115.0, 5.0, 1.0, 1.0, 0.0, 1.0, 31.0, 4.0, 212.0, 7.0, 3.0, 31.0, 2015.0]), label=6064.0),
 Row(features=DenseVector([3.0, 8138.0, 5.0, 1.0, 1.0, 0.0, 1.0, 31.0, 4.0, 212.0, 7.0, 3.0, 31.0, 2015.0]), label=8314.0),
 Row(features=DenseVector([6.0, 6330.0, 5.0, 1.0, 1.0, 0.0, 1.0, 31.0, 4.0, 212.0, 7.0, 3.0, 31.0, 2015.0]), label=5651.0),
 Row(features=DenseVector([7.0, 9875.0, 5.0, 1.0, 1.0, 0.0, 1.0, 31.0, 4.0, 212.0, 7.0, 3.0, 31.0, 2015.0]), label=15344.0),
 Row(features=DenseVector([8.0, 5776.0, 5.0, 1.0, 1.0, 0.0, 1.0, 31.0, 4.0, 212.0, 7.0, 3.0, 31.0, 2015.0]), label=8492.0)]